In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [2]:
# download a Postgres driver that will allow Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.17.jar

--2022-12-15 04:10:41--  https://jdbc.postgresql.org/download/postgresql-42.2.17.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1004734 (981K) [application/java-archive]
Saving to: ‘postgresql-42.2.17.jar’

postgresql-42.2.17. 100%[===================>] 981.19K  5.91MB/s    in 0.2s    

2022-12-15 04:10:41 (5.91 MB/s) - ‘postgresql-42.2.17.jar’ saved [1004734/1004734]



In [3]:
# start a Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VineReview").config("spark.driver.extraClassPath","/content/postgresql-42.2.17.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3P2HIOQCIN5ZU|          1|            0|          0|   N|                Y|
|R12VVR0WH5Q24V|          5|            0|          0|   N|                Y|
| RNCCKB6TV5EEF|          4|            0|          0|   N|                Y|
|R2NZXYIVCGB13W|          5|            0|          6|   N|                Y|
|R2EQ1TG9IT3OEQ|          3|            0|          0|   N|                Y|
|R1WXA9JSC2H1U1|          5|            1|          1|   N|                Y|
|R12ENYLFGGNWRV|          5|            1|          1|   N|                Y|
|R2R07E5PNXEUO5|          4|            0|          0|   N|                Y|
|R27BA52AKWMWN3|          5|            0|          0|   N|                Y|
| RLF8DOID2KD5O|          3|            0|          0|   N|     

In [41]:
# Retrieve all the rows where the total_votes count is equal to or greater than 20 
vine_votes_df=vine_df.filter("total_votes>=20")
vine_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [44]:
# Retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
votes_ratio_df=vine_votes_df.filter("helpful_votes/total_votes >= 0.5")
votes_ratio_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [46]:
# Retrieves all the rows where a review was written as part of the Vine program (paid)
paid_review_df=votes_ratio_df.filter(votes_ratio_df.vine == "Y")
paid_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2N45ZKRRZS856|          5|           21|         22|   Y|                N|
| R5OMLMK13A8NS|          5|           34|         38|   Y|                N|
|R2MPEQ4SPTEQNS|          4|          180|        184|   Y|                N|
| RIR0D3KJ0CQ31|          4|           21|         21|   Y|                N|
|R1SPWJDHUWWC5E|          5|           88|         98|   Y|                N|
|R1X6M5XA3FT98W|          5|           24|         26|   Y|                N|
|R37VCW6HA0Z72T|          5|           27|         28|   Y|                N|
|R2XRYNV2SY3ZKL|          5|           53|         56|   Y|                N|
|R1Y93KWKAX1P5N|          2|           26|         31|   Y|                N|
|R2QVTDYYLTP8SL|          5|           21|         24|   Y|     

In [60]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for paid program
from pyspark.sql.functions import count

# Total nb of reviews 
nb_paid_reviews= paid_review_df.count()
print("The total number of reviews for paid program is:")  
print(nb_paid_reviews)


The total number of reviews for paid program is:
22


In [59]:
# Nnumber of 5 stars reviews for paid program
nb_5star_paid_reviews = paid_review_df.filter(paid_review_df.star_rating == 5).count()
print("The total number of 5-star reviews for paid program is:")  
print(nb_5star_paid_reviews)


The total number of 5-star reviews for paid program is:
13


In [61]:
# Percentage of 5-star reviews for paid program
perc_5star_paid = float(nb_5star_paid_reviews)/float(nb_paid_reviews)
print("The percentage of 5-star reviews for paid program is:")  
print(perc_5star_paid)

The percentage of 5-star reviews for paid program is:
0.5909090909090909


In [62]:
# Retrieves all the rows where the review was not part of the Vine program (unpaid)
unpaid_review_df=votes_ratio_df.filter(votes_ratio_df.vine == "N")
unpaid_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [64]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for unpaid program
# Total nb of reviews 
nb_unpaid_reviews= unpaid_review_df.count()
print("The total number of reviews for unpaid program is:")  
print(nb_unpaid_reviews)

The total number of reviews for unpaid program is:
26987


In [65]:
# Nnumber of 5 stars reviews for unpaid program 
nb_5star_unpaid_reviews = unpaid_review_df.filter(unpaid_review_df.star_rating == 5).count()
print("The total number of 5-star reviews for unpaid program is:")  
print(nb_5star_unpaid_reviews)

The total number of 5-star reviews for unpaid program is:
14475


In [66]:
# Percentage of 5-star reviews for unpaid program
perc_5star_unpaid = float(nb_5star_unpaid_reviews)/float(nb_unpaid_reviews)
print("The percentage of 5-star reviews for unpaid program is:")  
print(perc_5star_unpaid)

The percentage of 5-star reviews for unpaid program is:
0.5363693630266425
